In [1]:
%defaultDatasource jdbc:h2:mem:db

# Navegando por Hierarquias

A seguir exercícios envolvendo navegação por hierarquias.

# Marcadores e Taxonomia

Considere o modelo de marcadores e taxonomia. Abaixo são apresentados modelos em Cypher e SQL.

![UML](marcadores-taxonomia-uml.png) ![Relacional](marcadores-taxonomia-er.png)

# Versão da hierarquia no Neo4J / Cypher

Entre na sandbox do Cypher (https://neo4j.com/sandbox-v2/?ref=hcard#) e execute os comandos (um bloco por vez).

## Criando um grafo de taxonomia e marcadores no Cypher

```
LOAD CSV WITH HEADERS FROM 'https://github.com/santanche/lab2learn/raw/master/data/bookmarks/taxonomia.csv' AS line
CREATE (:Categoria {id: line.categoria})

LOAD CSV WITH HEADERS FROM 'https://github.com/santanche/lab2learn/raw/master/data/bookmarks/taxonomia.csv' AS line
MATCH (cat:Categoria {id:line.categoria})
MATCH (sup:Categoria {id:line.superior})
CREATE (cat)-[:Superior]->(sup)

LOAD CSV WITH HEADERS FROM 'https://github.com/santanche/lab2learn/raw/master/data/bookmarks/marcadores.csv' AS line
MATCH (cat:Categoria {id:line.categoria})
CREATE (:Marcador {titulo:line.titulo,endereco:line.endereco,acessos:line.acessos})-[:Pertence]->(cat)

MATCH (m:Marcador)
MATCH (c:Categoria)
RETURN m, c
```

## Exercício 1

Escreva em Cypher uma consulta que retorne os marcadores da categoria Serviços.

## Exercício 2

Escreva um SQL aninhado para executar a mesma tarefa.

In [2]:
DROP TABLE IF EXISTS Taxonomia;
DROP TABLE IF EXISTS Marcadores;

CREATE TABLE Taxonomia (
  Categoria VARCHAR(50) NOT NULL,
  Superior  VARCHAR(50),
  PRIMARY KEY(Categoria),
  FOREIGN KEY(Superior)
    REFERENCES Taxonomia(Categoria)
      ON DELETE NO ACTION
      ON UPDATE CASCADE
);

CREATE TABLE Marcadores (
  Titulo VARCHAR(50) NOT NULL,
  Endereco VARCHAR(80) NOT NULL,
  Acessos INTEGER,
  Categoria VARCHAR(50) NOT NULL,
  PRIMARY KEY(Titulo),
  FOREIGN KEY(Categoria)
    REFERENCES Taxonomia(Categoria)
      ON DELETE NO ACTION
      ON UPDATE CASCADE
);

INSERT INTO Taxonomia VALUES ('Geral', NULL);
INSERT INTO Taxonomia VALUES ('Serviços', 'Geral');
INSERT INTO Taxonomia VALUES ('Acadêmico', 'Geral');
INSERT INTO Taxonomia VALUES ('Relacionamento', 'Geral');
INSERT INTO Taxonomia VALUES ('Busca', 'Serviços');
INSERT INTO Taxonomia VALUES ('Portal', 'Serviços');
INSERT INTO Taxonomia VALUES ('Mail', 'Serviços');
INSERT INTO Taxonomia VALUES ('Vendas', 'Serviços');
INSERT INTO Taxonomia VALUES ('Universidade', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('CG', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('Sociedade', 'Acadêmico');

INSERT INTO Marcadores VALUES ('Terra', 'http://www.terra.com.br', 295, 'Portal');
INSERT INTO Marcadores VALUES ('POVRay', 'http://www.povray.org', 2, 'CG');
INSERT INTO Marcadores VALUES ('SBC', 'http://www.sbc.org.br', 26, 'Sociedade');
INSERT INTO Marcadores VALUES ('Correios', 'http://www.correios.com.br', 45, 'Serviços');
INSERT INTO Marcadores VALUES ('GMail', 'http://www.gmail.com', 296, 'Mail');
INSERT INTO Marcadores VALUES ('Google', 'http://www.google.com', 1590, 'Busca');
INSERT INTO Marcadores VALUES ('Yahoo', 'http://www.yahoo.com', 134, 'Serviços');
INSERT INTO Marcadores VALUES ('Orkut', 'http://www.orkut.com', 45, 'Serviços');
INSERT INTO Marcadores VALUES ('iBahia', 'http://www.ibahia.com', 3, 'Portal');
INSERT INTO Marcadores VALUES ('Submarino', 'http://www.submarino.com.br', 320, 'Serviços');

SELECT * FROM Taxonomia;
SELECT * FROM Marcadores;

## Versão Cypher
```
```

In [3]:
// Versão SQL

# Medicamentos

Considere os medicamentos a seguir, suas indicações, patologias, efeitos colaterais e hierarquia de medicamentos.

A seguir há o modelo em Cypher e SQL dos mesmos.

## Exercício 3

Proponha uma consulta em Cypher que explore a relação entre medicamentos x efeitos colaterais x hierarquia de medicamentos (tam.
É possível executar esta consulta em SQL? Qual a dificuldade (opcional: escreva a consulta em SQL)

```
LOAD CSV WITH HEADERS FROM 'https://github.com/santanche/lab2learn/raw/master/data/faers-2017-dron/drug.csv' AS line
CREATE (:Drug {code: line.code, name: line.name})

LOAD CSV WITH HEADERS FROM 'https://github.com/santanche/lab2learn/raw/master/data/faers-2017-dron/drughierarchy.csv' AS line
MATCH (codedrug:Drug {code:line.codedrug})
MATCH (parent:Drug {code:line.parent})
CREATE (codedrug)-[:IsA]->(parent)

MATCH path=(:Drug {name: "VITAMIN D"})-[:IsA*]->(:Drug {name: "THING"})
RETURN path
```

In [4]:
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS Indication;
DROP TABLE IF EXISTS Person;
DROP TABLE IF EXISTS SideEffect;
DROP TABLE IF EXISTS DrugHierarchy;

CREATE TABLE Person(
    id INT NOT NULL,
    age INT,
    sex VARCHAR(1),
    PRIMARY KEY(Id)
) AS SELECT id, age, sex
FROM CSVREAD('../../data/faers-2017-dron/demo.csv');

CREATE TABLE Drug(
  code VARCHAR(200) NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT code, name
FROM CSVREAD('../../data/faers-2017-dron/drug.csv');

CREATE TABLE Pathology(
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT code, name
FROM CSVREAD('../../data/faers-2017-dron/pathology.csv');

CREATE TABLE Indication(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug VARCHAR(200) NOT NULL,
  FOREIGN KEY(idPerson)
    REFERENCES Person(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT idperson, codepathology, codedrug
FROM CSVREAD('../../data/faers-2017-dron/indication.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(idPerson)
    REFERENCES Person(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idperson, codepathology
FROM CSVREAD('../../data/faers-2017-dron/sideeffect.csv');

CREATE TABLE DrugHierarchy(
  codeDrug VARCHAR(200) NOT NULL,
  parent VARCHAR(200),
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(parent)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT codedrug, parent
FROM CSVREAD('../../data/faers-2017-dron/drughierarchy.csv');

SELECT * FROM Drug;
SELECT * FROM Pathology;
SELECT * FROM Indication;
SELECT * FROM Person;
SELECT * FROM SideEffect;
SELECT * FROM DrugHierarchy;